# 4. Vehicle trip details

> Takes a `vehicle_id` as input and returns route details of the current trip along with the live location of the vehicle.

## Sample request

```bash
curl 'https://bmtcmobileapi.karnataka.gov.in/WebAPI/VehicleTripDetails_v2' \
  -H 'Content-Type: application/json' \
  -H 'User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36' \
  --data-raw '{"vehicleId":21670}'
```

## Sample response

```json
{
    "RouteDetails": [
        {
            "rowid": 1,
            "tripid": 68043555,
            "routeno": "210-N",
            "routename": "KBS-UTH",
            "busno": "KA57F0614",
            "tripstatus": "Running",
            "tripstatusid": "1",
            "sourcestation": "Kempegowda Bus Station",
            "destinationstation": "Uttarahalli Bus Stand",
            "servicetype": "Non AC/Ordinary",
            "webservicetype": "Non-AC",
            "servicetypeid": 72,
            "lastupdatedat": "17-08-2025 12:33:18",
            "stationname": "Uttarahalli Bus Stand",
            "stationid": 22569,
            "actual_arrivaltime": null,
            "etastatus": "12:41",
            "etastatusmapview": "12:41",
            "latitude": 12.90535,
            "longitude": 77.54327,
            "currentstop": "",
            "laststop": "Gowdanapalya (Towards Uttarahalli)",
            "weblaststop": "Gowdanapalya",
            "nextstop": "Chikkallasandra Aralimara (Towards Uttarahalli)",
            "currlatitude": 12.911503,
            "currlongitude": 77.555923,
            "sch_arrivaltime": "12:48",
            "sch_departuretime": "12:48",
            "eta": "12:41",
            "actual_arrivaltime1": null,
            "actual_departudetime": null,
            "tripstarttime": "11:50",
            "tripendtime": "12:55",
            "routeid": 3796,
            "vehicleid": 21670,
            "responsecode": 200,
            "lastreceiveddatetimeflag": 1,
            "srno": 1584405201,
            "tripposition": 1,
            "stopstatus": 1,
            "stopstatus_distance": 1.53,
            "lastetaupdated": "2025-08-17T12:41:00",
            "minstopstatus_distance": 0.38
        }
    ],
    "LiveLocation": [
        {
            "latitude": 12.911503,
            "longitude": 77.555923,
            "location": "Gowdanapalya (Towards Kadirenahalli)",
            "lastrefreshon": "17-08-2025 12:33:18",
            "nextstop": "Chikkallasandra Aralimara (Towards Uttarahalli)",
            "previousstop": "Prarthana School (Towards Uttarahalli)",
            "vehicleid": 21670,
            "vehiclenumber": "KA57F0614",
            "routeno": "210-N",
            "servicetypeid": 72,
            "servicetype": "Non AC/Ordinary",
            "heading": 241.00,
            "responsecode": 200,
            "trip_status": 1,
            "lastreceiveddatetimeflag": 1
        }
    ],
    "Message": "Success",
    "Issuccess": true,
    "exception": null,
    "RowCount": 29,
    "responsecode": 200
}
```


# Data Issues

## Issue 1. Vehicle is assigned to more than one route

Vehicles on some occassions are assigned to more than one route. Ex: Vehicle `KA57F5808` is assigned to two routes, `routeno = 210-NA` and `routeno = D33-PPLO`.

```json
"LiveLocation": [
    {
        "latitude": 12.909809,
        "longitude": 77.536422,
        "location": "Depot-33 Poornapragna layout (Towards Depot-33 (Poornapragna layout))",
        "lastrefreshon": "17-09-2025 23:19:34",
        "nextstop": null,
        "previousstop": "Arehalli (Towards Kengeri)",
        "vehicleid": 27211,
        "vehiclenumber": "KA57F5808",
        "routeno": "210-NA",
        "servicetypeid": 72,
        "servicetype": "Non AC/Ordinary",
        "heading": 210.18,
        "responsecode": 200,
        "trip_status": 1,
        "lastreceiveddatetimeflag": 1
    },
    {
        "latitude": 12.909809,
        "longitude": 77.536422,
        "location": "Depot-33 Poornapragna layout (Towards Depot-33 (Poornapragna layout))",
        "lastrefreshon": "17-09-2025 23:19:34",
        "nextstop": null,
        "previousstop": "Arehalli (Towards Kengeri)",
        "vehicleid": 27211,
        "vehiclenumber": "KA57F5808",
        "routeno": "D33-PPLO",
        "servicetypeid": 72,
        "servicetype": "Non AC/Ordinary",
        "heading": 210.18,
        "responsecode": 200,
        "trip_status": 1,
        "lastreceiveddatetimeflag": 1
    }
]
```

## Issue 2. Live location is missing

Live location was empty for about `2.5%` of vehicles (`185` out of `7,247` vehicles) when run on `2025-09-17`. Ex: Vehicle ID  `28622` with registration number `KA01AR4181`.

```json
{
    "RouteDetails": [],
    "LiveLocation": [],
    "Message": "No Records Found",
    "Issuccess": true,
    "exception": null,
    "RowCount": 0,
    "responsecode": 200
}
```

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#| default_exp bmtc.apis.vehicle_trip_details

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import string
import json
import time
import datetime
from tqdm import tqdm
import geojson
import random

import requests
import pandas as pd
pd.set_option('display.max_columns', None)

from fastcore.all import Path
from nbdev.config import get_config

from traffic_data_bengaluru.utils import *
from traffic_data_bengaluru.bmtc.apis.vehicles import get_vehicles

In [ ]:
#| export
import logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s"
)
logger = logging.getLogger(__name__)

In [ ]:
#| hide
#| eval: false

data_directory = get_data_directory() / "bmtc"

# Fetch

In [ ]:
# | export
def fetch_vehicle_trip_details(vehicle_id: int, sleep_duration: float = 0.1):
    """Fetch trip details for a given vehicle ID from the BMTC API."""
    time.sleep(sleep_duration)
    url = "https://bmtcmobileapi.karnataka.gov.in/WebAPI/VehicleTripDetails_v2"

    headers = {
        "Content-Type": "application/json",
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36"
    }

    payload = json.dumps({"vehicleId": int(vehicle_id)})
    try:
        response = requests.post(url, headers=headers, data=payload)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
        print("Response text:", getattr(e.response, "text", None))
        return None

In [ ]:
#| hide
#| eval: false

directory = data_directory / 'raw' / 'trip_details' / str(int(datetime.datetime.now().timestamp()))
directory.mkdir(exist_ok=True, parents=True)

for index, row in tqdm(df_vehicles.iterrows(), total = df_vehicles.shape[0], desc = 'Fetching trip details'):
    trip_details = fetch_vehicle_trip_details(vehicle_id = row['vehicle_id']) 
    with open(directory / f"{row['vehicle_id']}.json", "w") as f:
        json.dump(trip_details, f, indent = 4)

# Extract

## Dataset 1. Live Location

In [ ]:
# | export
def extract_live_locations(trip_details):
    """Extract live location from trip details."""
    try:
        locations = trip_details['LiveLocation']
    except TypeError as e:
        # When trip_details is None.
        locations = []

    # When there are more than one live locations, it's mostly because of the vehicle assigned to more than one route at a time.
    # We could use the route details and live location to determine which is the right route that the vehicle is running on.
    return locations

In [ ]:
# | export
def collect_live_locations(directory: Path):
    """Extract live location for all trip details in a directory."""
    live_locations = []
    for filepath in tqdm(directory.ls(), total = directory.ls().__len__(), desc = 'Extracting live locations'):
        with open(filepath) as f:
            trip_details = json.load(f)
            
            # Extract live locations.
            live_locations += extract_live_locations(trip_details)
    live_locations = pd.DataFrame(live_locations)
    return live_locations

In [ ]:
#| hide
#| eval: false

directory = get_latest_directory(data_directory / 'raw' / 'vehicle_trip_details')
df_live_locations = collect_live_locations(directory)

filepath = data_directory / 'latest' / 'live_locations.csv'
filepath.parent.mkdir(parents=True, exist_ok=True)
df_live_locations.to_csv(filepath, index=False)

print(df_live_locations.shape)
df_live_locations.head(3)

Extracting live locations: 100%|██████████| 7235/7235 [00:02<00:00, 3254.74it/s]

(7093, 15)


,latitude,longitude,location,lastrefreshon,nextstop,previousstop,vehicleid,vehiclenumber,routeno,servicetypeid,servicetype,heading,responsecode,trip_status,lastreceiveddatetimeflag
0,12.778042,77.774879,Chikkanahalli Cross (Towards Chikkanahalli),24-09-2025 18:43:45,None,None,13232,KA01F9590,None,73,AC,111.00,200,NaN,1
1,13.027302,77.631906,"Hennur Main Road, Kachakaranahalli, Bengaluru ...",24-09-2025 19:18:14,Hennuru Junction (Towards Bagalur),Kacharakanahalli (Towards Hennur Bande),22996,KA57F5032,293-SD,72,Non AC/Ordinary,62.92,200,1.0,1
2,13.017702,77.555771,Depot-08 Yeshawanthapura (Towards Yashwanthapu...,24-09-2025 19:30:17,None,None,26486,KA57F5446,None,72,Non AC/Ordinary,287.00,200,NaN,1


## Dataset 2. Route Details

In [ ]:
# | export
def extract_route_detail(trip_detail):
    try:
        details = trip_detail['RouteDetails']
    except TypeError as e:
        details = []
    return details

In [ ]:
#| hide
#| eval: false
def collect_route_details(directory: Path):
    route_details = []
    for filepath in tqdm(directory.ls(), total = directory.ls().__len__(), desc = 'Extracting route details'):
        with open(filepath) as f:
            trip_detail = json.load(f)
            
            route_detail = extract_route_detail(trip_detail)
            route_details += route_detail
    route_details = pd.DataFrame(route_details)
    return route_details

In [ ]:
#| hide
#| eval: false

directory = get_latest_directory(data_directory / 'raw' / 'vehicle_trip_details')
df_route_details = collect_route_details(directory)

filepath = data_directory / 'latest' / 'route_details.csv'
filepath.parent.mkdir(parents=True, exist_ok=True)
df_route_details.to_csv(filepath, index=False)

print(df_route_details.shape)
df_route_details.head(3)

Extracting route details: 100%|██████████| 7235/7235 [00:02<00:00, 3152.95it/s]


(155273, 43)


,rowid,tripid,routeno,routename,busno,tripstatus,tripstatusid,sourcestation,destinationstation,servicetype,...,routeid,vehicleid,responsecode,lastreceiveddatetimeflag,srno,tripposition,stopstatus,stopstatus_distance,lastetaupdated,minstopstatus_distance
0,1,71096918,293-SD,SBS-DVH,KA57F5032,Running,1,Shivajinagara Bus Station,Devanahalli Old Bus Stand,Non AC/Ordinary,...,14678,22996,200,1,1659389086,1,2,999.0,None,0.15
1,1,71096918,293-SD,SBS-DVH,KA57F5032,Running,1,Shivajinagara Bus Station,Devanahalli Old Bus Stand,Non AC/Ordinary,...,14678,22996,200,1,1659389087,1,0,999.0,None,0.15
2,1,71096918,293-SD,SBS-DVH,KA57F5032,Running,1,Shivajinagara Bus Station,Devanahalli Old Bus Stand,Non AC/Ordinary,...,14678,22996,200,1,1659389088,1,0,999.0,None,0.15


In [ ]:
#| hide
#| eval: false

sample_vehicleid = df_live_locations[df_live_locations['vehicleid'].notna()].sample(1).iloc[0]['vehicleid']
columns = ['busno', 'stationname', 'sch_arrivaltime', 'actual_arrivaltime']
df = df_route_details[df_route_details['vehicleid'] == sample_vehicleid].reset_index(drop=True)[columns]

# Keep the raw values as is, and calculate diffs on the fly when needed
def calc_time_diff(series):
    times = pd.to_datetime(series, format="%H:%M", errors="coerce")
    return times.diff().dt.total_seconds()

df["sch_diff"] = calc_time_diff(df["sch_arrivaltime"])
df["actual_diff"] = calc_time_diff(df["actual_arrivaltime"])

df

,busno,stationname,sch_arrivaltime,actual_arrivaltime,sch_diff,actual_diff
0,KA57F6564,Banashankari Bus Station,19:40,19:22,NaN,NaN
1,KA57F6564,Sangam Circle,19:43,19:23,180.0,60.0
2,KA57F6564,Jayanagara 40th Cross,19:45,19:27,120.0,240.0
3,KA57F6564,Jayanagara 38th Cross,19:46,19:29,60.0,120.0
4,KA57F6564,Jayanagara Telephone Exchange,19:47,19:30,60.0,60.0
5,KA57F6564,Jayanagara Bus Station,19:55,19:36,480.0,360.0
6,KA57F6564,3rd Block Jayanagara,19:59,19:38,240.0,120.0
7,KA57F6564,Madhavan Park,20:01,19:39,120.0,60.0
8,KA57F6564,Rani Saraladevi High School,20:02,19:42,60.0,180.0
9,KA57F6564,Ashoka Pillar,20:03,19:44,60.0,120.0


# Task

In [ ]:
# | export
def task_fetch_vehicle_trip_details(data_directory: Path):
    filename = f'{str(int(datetime.datetime.now().timestamp()))}'
    directory = data_directory / 'raw' / 'vehicle_trip_details' / filename
    directory.mkdir(exist_ok=True, parents=True)

    logger.info("Fetching vehicle trip details ...")
    df_vehicles = get_vehicles(data_directory)
    for index, row in tqdm(df_vehicles.iterrows(), total = df_vehicles.shape[0], desc = 'Fetching vehicle trip details'):
        trip_details = fetch_vehicle_trip_details(vehicle_id = row['vehicle_id']) 
        with open(directory / f"{row['vehicle_id']}.json", "w") as f:
            json.dump(trip_details, f, indent = 4)

In [ ]:
#| hide
#| eval: false

task_fetch_vehicle_trip_details(data_directory=data_directory)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()